## Búsquedas que coinciden con reservas

In [2]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [60]:
b = pd.read_csv("/home/dsc/Repositories/Master-in-Data-Science/bookings.csv.bz2",
                sep="^",
                nrows=1e5,
# Hora de embarque, aeropuertdo de embarque, aeropuerto de destino
                usecols=["brd_time           ","brd_port","off_port"],
                parse_dates=["brd_time           "]
               )

In [16]:
b.columns

Index([u'brd_port', u'off_port', u'brd_time           '], dtype='object')

In [61]:
# Remombramos la columna con espacios
b["date"]=b["brd_time           "]

In [62]:
b = b.drop("brd_time           ",axis=1)

In [71]:
b.date

0       2013-03-07 08:50:00
1       2013-04-12 13:04:00
2       2013-07-15 07:00:00
3       2013-04-24 23:59:00
4       2013-05-14 20:15:00
5       2013-04-07 10:23:00
6       2013-04-11 11:15:00
7       2013-04-14 11:05:00
8       2013-04-16 15:45:00
9       2013-04-18 08:00:00
10      2013-04-19 22:25:00
11      2013-04-14 11:05:00
12      2013-04-16 15:45:00
13      2013-04-18 08:00:00
14      2013-04-19 22:25:00
15      2013-04-14 11:05:00
16      2013-04-16 15:45:00
17      2013-04-18 08:00:00
18      2013-04-19 22:25:00
19      2013-04-06 13:28:00
20      2013-05-25 07:25:00
21      2013-05-25 08:30:00
22      2013-03-24 20:50:00
23      2013-08-05 12:35:00
24      2013-08-05 12:35:00
25      2013-08-22 10:35:00
26      2013-03-24 19:15:00
27      2013-03-24 19:15:00
28      2013-03-24 20:50:00
29      2013-03-24 19:15:00
                ...        
99970   2013-03-21 19:30:00
99971   2013-03-21 19:30:00
99972   2013-03-21 15:30:00
99973   2013-03-21 12:35:00
99974   2013-03-26 1

In [42]:
s = pd.read_csv("/home/dsc/Repositories/Master-in-Data-Science/searches.csv.bz2",
                sep="^",
                nrows=1e5,
                usecols=["Seg1Date","Origin","Destination"],
                parse_dates=["Seg1Date"]
               )

In [73]:
s.head()

,Origin,Destination,Seg1Date
0,TXL,AUH,2013-01-26
1,ATH,MIL,2013-01-04
2,ICT,SFO,2013-08-02
3,RNB,ARN,2013-01-02
4,OSL,MAD,2013-03-22


In [63]:
# Cambiamos el formato de la columna date de b
b["date2"]=b.date.apply(lambda d: d.date())

In [74]:
s["date2"]=s.Seg1Date.apply(lambda d: d.date())

In [77]:
# Left Join de ambos df para mantener todos los datos y poder comparar con el total
join = pd.merge(
        s,b,
        how="left",
        left_on=["Origin","Destination","date2"],
        right_on=["brd_port","off_port","date2"])

In [82]:
join.head()

,Origin,Destination,Seg1Date,date2,brd_port,off_port,date
0,TXL,AUH,2013-01-26,2013-01-26,NaN,NaN,NaT
1,ATH,MIL,2013-01-04,2013-01-04,NaN,NaN,NaT
2,ICT,SFO,2013-08-02,2013-08-02,NaN,NaN,NaT
3,RNB,ARN,2013-01-02,2013-01-02,NaN,NaN,NaT
4,OSL,MAD,2013-03-22,2013-03-22,NaN,NaN,NaT


In [86]:
# Añadimos una columna que nos dice si cruza ambas tablas
join["match"] = join.brd_port.apply(lambda b: not pd.isnull(b))

In [87]:
join.head()

,Origin,Destination,Seg1Date,date2,brd_port,off_port,date,match
0,TXL,AUH,2013-01-26,2013-01-26,NaN,NaN,NaT,False
1,ATH,MIL,2013-01-04,2013-01-04,NaN,NaN,NaT,False
2,ICT,SFO,2013-08-02,2013-08-02,NaN,NaN,NaT,False
3,RNB,ARN,2013-01-02,2013-01-02,NaN,NaN,NaT,False
4,OSL,MAD,2013-03-22,2013-03-22,NaN,NaN,NaT,False


In [89]:
join.groupby("match").count()

,Origin,Destination,Seg1Date,date2,brd_port,off_port,date
match,,,,,,,
False,100000,100000,99605,100000,0,0,0


In [ ]:
# Hay nulos en la columna Seg1Date. Podríamos quitarlos al crear s, añadiendo 
# s.dropna(),
# b.dropna()

# Hay espacios en los nombres de los aeropuertos, habría que quitarlos